# WHAT IS IMPORTANT ABOUT THIS NOTEBOOK?

* The dataset had unbalanced contributions (Some patients has many more OCT scans than other patiens)
* The code of this notebook allows to select 14 OCT for each patient, 7 of each eye when it is possible
* The resulting datatest selected can be found in C:\Users\line\Desktop\Mauro\3_DataSet\OCT_balanced\amdP and C:\Users\line\Desktop\Mauro\3_DataSet\OCT_balanced\controlP
* You can simply create a dataset, composed of X,y numpy array by calling the getBalancedXYData function imported from buildDataset as shown in the last cell

# NO NEED TO RUN THIS CODE AGAIN
## Here we make sure we keep only one 14 OCT's per patient
1) You can see also why we chosed 14 as a cutoff number in the following code
2) We drop one patient since he has only 2 oct scans
3) We sample randomly from the OCT's and keep, when possible, 7 scan for every eye
4) We dedicate 1 control and 1 amd patients to the validation set: this allows for 12CTRL,6AMD, that allows for 2,3,4,6 fold cross validation.

In [3]:
from buildDataset import *
%matplotlib inline
import random

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,ConfusionMatrixDisplay,f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
root_path =      DATAPATHS["preprocessed"]
from sklearn.model_selection import KFold


In [55]:
# The only take out from this function are the printings below, describing how many OCT per patient and per eye
def checkNditribution(root_path):
    '''Checking the number of OCT scans of each patient to be able to select equal contribution from each patient'''
    def fix(ls):
        ls2 = []
        if '+' in ls:
            ls2.append( os.path.split(ls.split("+")[0])[1].split(".")[0] ) 
            ls2.append( os.path.split(ls.split("+")[1])[1].split(".")[0] )
        else:
            ls2.append(os.path.split(ls)[1].split(".")[0])
        return (" ").join(ls2)    

    paths =  glob.glob(os.path.join(root_path + '\*.pickle'))
    paths_amd = []
    paths_control = []
    num_amd = []
    num_control = []
    rateoAmd = []
    rateoControl = []
    total_amd = 0
    total_control = 0
    iterator = iter(range(0,len(paths)))
    AMDtwoEyesSamePatient = []
    CtrltwoEyesSamePatient = []

    # We need to know how many OCT per patient, but the paths are per eye, hence we need to perform some checks on 
    # the path name, additionally not all patiens have two paths, since for one patient only one eye was imaged
    for i in iterator:
        group = paths[i].split("\\").pop().split("_")[0]
        # Check if the two next paths are of the same patient (left and right eye), if the case we treat them as one single path,
        # and use a + to concatenate the two paths, then the fix function will be able to split the path back into two paths.
        two_eyes_same_patient = paths[i].split("\\")[-1].split("_")[:-1]==paths[i+1].split("\\")[-1].split("_")[:-1]
        if(two_eyes_same_patient):
            if group == 'amd':
                AMDtwoEyesSamePatient.append((len(pd.read_pickle(paths[i])),len(pd.read_pickle(paths[i + 1]))))
            else:
                CtrltwoEyesSamePatient.append((len(pd.read_pickle(paths[i])),len(pd.read_pickle(paths[i + 1]))))
            PPocts = pd.read_pickle(paths[i]) + pd.read_pickle(paths[i + 1]) 
            path = paths[i] + '+' + paths[i+1]
            next(iterator, None)
            print(i,fix(path),len(pd.read_pickle(paths[i])),len(pd.read_pickle(paths[i + 1])) )
        else:
            PPocts =pd.read_pickle(paths[i])
            path = paths[i]
            print(i,fix(path),len(pd.read_pickle(paths[i])))
        
        num_octs = len(PPocts)
        rateos = [np.sum(PPoct.label())/(PPoct.label().shape[0]) for PPoct in PPocts]
        rateo = np.mean(rateos)
        if (group == 'amd'):
            paths_amd.append(path)
            num_amd.append(num_octs)
            total_amd += num_octs
            rateoAmd.append(rateo) 
        elif (group == 'control'):
            paths_control.append(path)
            num_control.append(num_octs)
            total_control += num_octs
            rateoControl.append(rateo) 
    print(f'the paths cntrl are: {len(paths_control)} with associated nocts{num_control}, the paths amd are: with associated nocts{num_amd}')
    print(f'the rateos for control are {rateoControl} and for amd are {rateoAmd}')
    print(f'the pairs are for control {len(CtrltwoEyesSamePatient)} , for AMD {AMDtwoEyesSamePatient}')

checkNditribution(root_path)

0 amd_0_left amd_0_right 9 49
2 amd_1_left amd_1_right 36 12
4 amd_2_left amd_2_right 27 68
6 amd_3_left amd_3_right 45 62
8 amd_4_left amd_4_right 22 38
10 amd_5_left 19
11 amd_6_left amd_6_right 97 66
13 control_0_left control_0_right 60 14
15 control_10_left control_10_right 16 13
17 control_11_left control_11_right 8 7
19 control_12_left control_12_right 5 9
21 control_1_left control_1_right 0 2
23 control_2_left control_2_right 15 27
25 control_3_left control_3_right 10 10
27 control_4_left control_4_right 10 12
29 control_5_left control_5_right 15 13
31 control_6_left control_6_right 17 4
33 control_7_left control_7_right 57 29
35 control_8_left control_8_right 8 9
37 control_9_left control_9_right 26 37
the paths cntrl are: 13 with associated nocts[74, 29, 15, 14, 2, 42, 20, 22, 28, 21, 86, 17, 63], the paths amd are: with associated nocts[58, 48, 95, 107, 60, 19, 163]
the rateos for control are [0.30310388513513514, 0.04660560344827587, 0.04939236111111111, 0.037109375, 0.05924

In [56]:
# Number of OCTs per patient and per eye
ctrlPairs = [(60, 14), (16, 13), (8, 7), (5, 9), (0, 2), (15, 27), (10, 10), (10, 12), (15, 13), (17, 4), (57, 29), (8, 9), (26, 37)]
AMDPairs = [(9, 49), (36, 12), (27, 68), (45, 62), (22, 38), (19,0), (97, 66)]

# Hence we want only 14 images per patient to have an equal contribution

In [223]:
def train_test_path_split(root_path,p = 0.8):
    def keep_good_segmentation(PPocts,special = False):
        "Filter data to avoid the issue of wrong segmentation"
        filtered = []
        for PPoct in PPocts:
            if PPoct.image().shape[0] < 120:
                filtered.append(PPoct)
            elif special:
                filtered.append(PPoct)      
        return filtered

       
    paths =  glob.glob(os.path.join(root_path + '\*.pickle'))
    paths_amd = []
    paths_control = []
    num_amd = []
    num_control = []
    rateoAmd = []
    rateoControl = []
    total_amd = 0
    total_control = 0
    iterator = iter(range(0,len(paths)))
    AMDtwoEyesSamePatient = []
    CtrltwoEyesSamePatient = []

    
    for i in iterator:
        group = paths[i].split("\\").pop().split("_")[0]
        Id = paths[i].split("\\").pop().split("_")[1]
        special = group != 'amd' and Id == '12'
        # Check if the two next paths are of the same patient (left and right eye), if the case we treat them as one
        two_eyes_same_patient = paths[i].split("\\")[-1].split("_")[:-1]==paths[i+1].split("\\")[-1].split("_")[:-1]
        if(two_eyes_same_patient):
            if group == 'amd':
                AMDtwoEyesSamePatient.append((len(keep_good_segmentation(pd.read_pickle(paths[i]),special)),len(keep_good_segmentation(pd.read_pickle(paths[i + 1]),special))))
            else:
                CtrltwoEyesSamePatient.append((len(keep_good_segmentation(pd.read_pickle(paths[i]),special)),len(keep_good_segmentation(pd.read_pickle(paths[i + 1]),special))))
            PPocts = keep_good_segmentation(pd.read_pickle(paths[i]) + pd.read_pickle(paths[i + 1]))
            path = paths[i] + '+' + paths[i+1]
            next(iterator, None)

        else:
            PPocts =keep_good_segmentation(pd.read_pickle(paths[i]))
            path = paths[i]
        print(i,path,len(pd.read_pickle(paths[i])),len(pd.read_pickle(paths[i + 1])) )
        num_octs = len(PPocts)
        rateos = [np.sum(PPoct.label())/(PPoct.label().shape[0]) for PPoct in PPocts]
        rateo = np.mean(rateos)
        if (group == 'amd'):
            paths_amd.append(path)
            num_amd.append(num_octs)
            total_amd += num_octs
            rateoAmd.append(rateo) 
        elif (group == 'control'):
            paths_control.append(path)
            num_control.append(num_octs)
            total_control += num_octs
            rateoControl.append(rateo) 
    print(f'the paths cntrl are: {paths_control[4]} with associated nocts{num_control}, the paths amd are: with associated nocts{num_amd}')
    print(f'the rateos for control are {rateoControl} and for amd are {rateoAmd}')
    print(f'the pairs are for control {CtrltwoEyesSamePatient} , for AMD {AMDtwoEyesSamePatient}')

train_test_path_split(root_path,p = 0.8)

0 C:\Users\line\Desktop\Mauro\preprocessing\preprocessed_octs\amd_0_left.pickle+C:\Users\line\Desktop\Mauro\preprocessing\preprocessed_octs\amd_0_right.pickle 9 49
2 C:\Users\line\Desktop\Mauro\preprocessing\preprocessed_octs\amd_1_left.pickle+C:\Users\line\Desktop\Mauro\preprocessing\preprocessed_octs\amd_1_right.pickle 36 12
4 C:\Users\line\Desktop\Mauro\preprocessing\preprocessed_octs\amd_2_left.pickle+C:\Users\line\Desktop\Mauro\preprocessing\preprocessed_octs\amd_2_right.pickle 27 68
6 C:\Users\line\Desktop\Mauro\preprocessing\preprocessed_octs\amd_3_left.pickle+C:\Users\line\Desktop\Mauro\preprocessing\preprocessed_octs\amd_3_right.pickle 45 62
8 C:\Users\line\Desktop\Mauro\preprocessing\preprocessed_octs\amd_4_left.pickle+C:\Users\line\Desktop\Mauro\preprocessing\preprocessed_octs\amd_4_right.pickle 22 38
10 C:\Users\line\Desktop\Mauro\preprocessing\preprocessed_octs\amd_5_left.pickle 19 97
11 C:\Users\line\Desktop\Mauro\preprocessing\preprocessed_octs\amd_6_left.pickle+C:\Users

In [7]:
import os
# We keep only 30 % of the original data but sample randomly from the oct scans to ensure more vairability
# Let's create a dict with the numbers of imgs and the paths
order = [0,10,11,12,1,2,3,4,5,6,7,8,9]
basepath = r'C:\Users\line\Desktop\Mauro\preprocessing\preprocessed_octs'  # amd_0_left.pickle
ctrlPairs = [(60, 14), (16, 13), (8, 7), (5, 9), (0, 2), (15, 27), (10, 10), (10, 12), (15, 13), (17, 4), (57, 29), (7, 7), (26, 37)]
AMDPairs = [(9, 49), (36, 12), (27, 67), (45, 59), (22, 38), (93, 63)]

ctrlPairs2 = [(7,7) if (pair[0] >6 and pair[1] > 6) else pair for pair in ctrlPairs]
AMDPairs2 = [(7,7) if (pair[0] >6 and pair[1] > 6) else pair for pair in AMDPairs]
#FIX PAIRS WHERE AT LEAST ONE IS SMALLER THAN 7
# Patient control9 is 10,4 since has only 4 octs for the right eye
ctrlPairs2[9] = (10,4)
# Patient control2 is dropped since it only has 2 OCT scans
ctrlPairs2[4] = (0,0)
# Patient AMD5 has only left eye and was not present in the pairs
AMDPairs2[5] = (14,0)
AMDPairs2.append((7,7))

#PathToNum = {path: value for (key, value) in iterable}
ctrlPaths = [(os.path.join(basepath,'control_'+ str(i) +'_left.pickle'),
              os.path.join(basepath,'control_'+ str(i) +'_right.pickle')) for i in order]
AMDPaths = [(os.path.join(basepath,'amd_'+ str(i) +'_left.pickle'),
              os.path.join(basepath,'amd_'+ str(i) +'_right.pickle')) for i in range(len(AMDPairs2))]

In [8]:
dictLeftCtrl = {quad[0][0]:quad[1][0]  for quad in list(zip(ctrlPaths,ctrlPairs2))}   
dictRightCtrl ={quad[0][1]:quad[1][1]  for quad in list(zip(ctrlPaths,ctrlPairs2))} 
dictCtrl = dictLeftCtrl | dictRightCtrl
print(dictCtrl)

{'C:\\Users\\line\\Desktop\\Mauro\\preprocessing\\preprocessed_octs\\control_0_left.pickle': 7, 'C:\\Users\\line\\Desktop\\Mauro\\preprocessing\\preprocessed_octs\\control_10_left.pickle': 7, 'C:\\Users\\line\\Desktop\\Mauro\\preprocessing\\preprocessed_octs\\control_11_left.pickle': 7, 'C:\\Users\\line\\Desktop\\Mauro\\preprocessing\\preprocessed_octs\\control_12_left.pickle': 5, 'C:\\Users\\line\\Desktop\\Mauro\\preprocessing\\preprocessed_octs\\control_1_left.pickle': 0, 'C:\\Users\\line\\Desktop\\Mauro\\preprocessing\\preprocessed_octs\\control_2_left.pickle': 7, 'C:\\Users\\line\\Desktop\\Mauro\\preprocessing\\preprocessed_octs\\control_3_left.pickle': 7, 'C:\\Users\\line\\Desktop\\Mauro\\preprocessing\\preprocessed_octs\\control_4_left.pickle': 7, 'C:\\Users\\line\\Desktop\\Mauro\\preprocessing\\preprocessed_octs\\control_5_left.pickle': 7, 'C:\\Users\\line\\Desktop\\Mauro\\preprocessing\\preprocessed_octs\\control_6_left.pickle': 10, 'C:\\Users\\line\\Desktop\\Mauro\\preprocessi

In [9]:
dictLeftAMD = {quad[0][0]:quad[1][0]  for quad in list(zip(AMDPaths,AMDPairs2))}   
dictRightAMD ={quad[0][1]:quad[1][1]  for quad in list(zip(AMDPaths,AMDPairs2))} 
dictAMD = dictLeftAMD | dictRightAMD
print(dictAMD)

{'C:\\Users\\line\\Desktop\\Mauro\\preprocessing\\preprocessed_octs\\amd_0_left.pickle': 7, 'C:\\Users\\line\\Desktop\\Mauro\\preprocessing\\preprocessed_octs\\amd_1_left.pickle': 7, 'C:\\Users\\line\\Desktop\\Mauro\\preprocessing\\preprocessed_octs\\amd_2_left.pickle': 7, 'C:\\Users\\line\\Desktop\\Mauro\\preprocessing\\preprocessed_octs\\amd_3_left.pickle': 7, 'C:\\Users\\line\\Desktop\\Mauro\\preprocessing\\preprocessed_octs\\amd_4_left.pickle': 7, 'C:\\Users\\line\\Desktop\\Mauro\\preprocessing\\preprocessed_octs\\amd_5_left.pickle': 14, 'C:\\Users\\line\\Desktop\\Mauro\\preprocessing\\preprocessed_octs\\amd_6_left.pickle': 7, 'C:\\Users\\line\\Desktop\\Mauro\\preprocessing\\preprocessed_octs\\amd_0_right.pickle': 7, 'C:\\Users\\line\\Desktop\\Mauro\\preprocessing\\preprocessed_octs\\amd_1_right.pickle': 7, 'C:\\Users\\line\\Desktop\\Mauro\\preprocessing\\preprocessed_octs\\amd_2_right.pickle': 7, 'C:\\Users\\line\\Desktop\\Mauro\\preprocessing\\preprocessed_octs\\amd_3_right.pickl

In order to run the cell below:
1) REMOVE 5_left.pickle from the amd folder
2) uncomment and run the cell below
3) Put amd back in the amd folder and add a copy named 5.pickle to the amdP folder

In [12]:
#Be carefull to amd 5 !!!
# generation of the balance dataset:
basepath = r'C:\Users\line\Desktop\Mauro\3_DataSet\OCT_balanced'
inamd = os.path.join(basepath,'amd')
outamd = os.path.join(basepath,'amdP')
incontrol = os.path.join(basepath,'control')
outcontrol = os.path.join(basepath,'controlP')

def toSinglePatient(inpath,outpathbase):
    paths =  glob.glob(os.path.join(inpath + '\*.pickle'))

    for i in range(0,len(paths),2):
        path = paths[i]
        nextpath = paths[i+1]
        Id = os.path.split(path)[-1].split("_")[0]
        outpath = os.path.join(outpathbase,Id + '.pickle')
        # Read content of the two files
        PPocts = pd.read_pickle(path) + pd.read_pickle(nextpath)
        #put into output file
        with open(outpath, 'wb') as f:
                pickle.dump(PPocts, f)
                
# BE CAREFUL TO AMD5 
# toSinglePatient(inamd,outamd)
# toSinglePatient(incontrol,outcontrol)

# Now we have our balanced - contribution dataset
each folder is containing the data of a given user, hence we can cross validate on a patient level =)

In [4]:
# Creation of NP arrays X and y
import cv2

pathsamd = r'C:\Users\line\Desktop\Mauro\3_DataSet\OCT_balanced\amdP'
pathscontrol = r'C:\Users\line\Desktop\Mauro\3_DataSet\OCT_balanced\controlP'
def getXYdata(paths, mode,rootpath = None,normmode = 'Z-score'):
    def normalize(img,normmode = 'Z-score'):
        if normmode == 'Z-score':
            img = dataset.z_score(img)
        elif normmode == 'EQ-hist': 
            img = exposure.equalize_hist(img)
        elif normmode == 'CLAHE':
            #TO Implement normalization by zone
            # Adaptive Equalization
            img = exposure.equalize_adapthist(img/max((img.max(),-img.min())), clip_limit=0.03)            
        return img
    
    def getxmin(rootpath):
        themin = 100000
        allpaths =  glob.glob(os.path.join(rootpath,'amdP' + '\*.pickle')) + glob.glob(os.path.join(rootpath,'controlP' + '\*.pickle'))
        for path in allpaths:
            PPocts = pd.read_pickle(path)
            for PPoct in PPocts:
                xnew = PPoct.image()
                if(xnew.shape[0] < themin):
                    themin = xnew.shape[0]
        return themin
    
    if mode == 'thickness':
        xmin = 5 
    elif mode =='raw':
        xmin = getxmin(rootpath)
        
    X = np.empty((0,xmin))
    y = np.empty((0,))
    
    for path in paths:
        PPocts = pd.read_pickle(path)
        for PPoct in PPocts:
            if mode == 'thickness':
                xnew = np.transpose(PPoct.thicknesses())
            elif mode == 'raw':
                img =PPoct.image()
                if img.shape[0] > 120:
                    img = img[-100::,:]
                    
                xnew = np.transpose(normalize(img,normmode = normmode))
                xnew = cv2.resize(xnew, dsize=(xmin,768 ))
            ynew = PPoct.label()
            #stack vertically the x and y
            X = np.vstack((X,xnew))
            y = np.hstack((y,ynew))        
      
    return X,y 


In [14]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=6, shuffle=False)

In [16]:
paths = glob.glob(os.path.join(rootpath,'controlP' + '\*.pickle'))
print(len(paths))
for path in paths:
    print(len(pd.read_pickle(path)))

12
14
14
14
14
14
14
14
14
14
14
14
14


In [18]:
paths = glob.glob(os.path.join(rootpath,'amdP' + '\*.pickle'))
print(len(paths))
for path in paths:
    print(len(pd.read_pickle(path)))

6
14
14
14
14
14
14


In [5]:
rootpath = r'C:\Users\line\Desktop\Mauro\3_DataSet\OCT_balanced'
paths = glob.glob(os.path.join(rootpath,'controlP' + '\*.pickle'))
paths2 = glob.glob(os.path.join(rootpath,'amdP' + '\*.pickle'))
merged = [[paths[2*i],paths[2*i+1],paths2[i]]  for i in range(len(paths2))]
flattenedmerged = [item for sublist in merged for item in sublist]
print(flattenedmerged)

['C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\controlP\\0.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\controlP\\10.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\amdP\\0.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\controlP\\11.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\controlP\\12.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\amdP\\1.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\controlP\\2.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\controlP\\3.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\amdP\\2.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\controlP\\4.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\controlP\\5.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\amdP\\3.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_bal

In [ ]:
flattenedmerged = ['C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\controlP\\0.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\controlP\\10.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\amdP\\0.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\controlP\\11.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\controlP\\12.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\amdP\\1.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\controlP\\2.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\controlP\\3.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\amdP\\2.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\controlP\\4.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\controlP\\5.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\amdP\\3.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\controlP\\6.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\controlP\\7.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\amdP\\5.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\controlP\\8.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\controlP\\9.pickle', 'C:\\Users\\line\\Desktop\\Mauro\\3_DataSet\\OCT_balanced\\amdP\\6.pickle']

In [6]:
X,y = getXYdata(flattenedmerged, mode = 'raw',rootpath = rootpath,normmode = 'EQ-hist')
print(X.shape)
print(y.shape)
print(768*14*18)

(193536, 70)
(193536,)
193536


# WHAT IS IMPORTANT ABOUT THIS NOTEBOOK?

* The dataset had unbalanced contributions (Some patients has many more OCT scans than other patiens)
* The code of this notebook allows to select 14 OCT for each patient, 7 of each eye when it is possible
* The resulting datatest selected can be found in C:\Users\line\Desktop\Mauro\3_DataSet\OCT_balanced\amdP and C:\Users\line\Desktop\Mauro\3_DataSet\OCT_balanced\controlP
* You can simply create a dataset, composed of X,y numpy array by calling the getBalancedXYData function imported from buildDataset as shown below

In [4]:
X,y = getBalancedXYData(mode = 'thickness',normmode = '')
print(X.shape)
print(y.shape)
print(768*14*18)

(193536, 5)
(193536,)
193536
